## Introduction



We're thinking about the problem of finding the cheapest possible
nutritious diet. As we have discussed, this can be set up as a linear program.

$$
    \min_x p'x
n$$

such that
$$
Ax \geq b
$$

where $p$ is a vector of prices, $A$ is a matrix that maps
vectors of quantities of food into vectors of nutrients, and where b is a vector of nutrient constraints.

The goal of this notebook is to expose the class to some data sources and to generate a minimum cost diet for the United States using recent national average price data.



## Food and Nutrient Data



The USDA maintains a database of foods called the [FNDDS](https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds/) (Food and Nutrition Database for Dietary Studies). In it, they define "recipes," which are made up of "ingredients." There are one or more ingredients per recipe. Ingredients are then mapped to nutrients in terms of nutrient units per 100 g of an ingredient. Thus, using these mappings, we can generate a FNDDS recipe-level food-to-nutrient matrix we can use as our "A" matrix.

Lets first define a helper function we will use to make sure ids are formatted similarly (to help with dataframe merging). 



In [1]:
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/12Z4n8HbFZRYvH6B-D8EDLDibRiL50zNMlSBLMJ41C1o/"

Lets load in the recipes and see what we have.



In [1]:
import pandas as pd
from eep153_tools.sheets import read_sheets

recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))


# lets see an example of a recipe.. note 
recipes[recipes["recipe"].str.contains("Vegetable lasagna", case=False)]

And we can take a look at the ingredients. I'll call this dataframe "nutrition" because that's what we really care about in here.



In [1]:
nutrition = (read_sheets(url, sheet="nutrients")
             .assign(ingred_code = lambda df: df["ingred_code"].apply(format_id)))

display(nutrition.head())
nutrition.columns

Lets merge these mappings to end up with a matrix of recipes and their nutrients as measured by units of a nutrient per 100 grams of that food.



In [1]:
# normalize weights to percentage terms. 
recipes['ingred_wt'] = recipes['ingred_wt']/recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df = recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df[numeric_cols] = df[numeric_cols].mul(df["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. The ** effectively "unpacks" the key value pairs in each dictionary
df = df.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "recipe": "first"})

df.index.name = "recipe_id"

food_names = df["recipe"]

If we recall, the $ A  $ matrix maps foods into constrained nutrients. This is effectively the transpose of our `df`. We'll have to do more before we create `A` - not all of these nutrients are constrained, and unfortunately, we won't have prices for all foods.



## Prices



Now that we have the bones of the matrix $ A $, lets next consider the price vector $ p $. The USDA generates national average prices ([Purchase to Plate](https://www.ers.usda.gov/data-products/purchase-to-plate)) for these FNDDS foods using scanner data from grocery stores all over the country. These are in USD per 100 grams of a recipe. They have been doing this for a while, and they produce the prices in two-year batches.



In [1]:
prices = read_sheets(url, sheet="prices")[["food_code", "year", "price"]]

prices["food_code"] = prices["food_code"].apply(format_id)

prices = prices.set_index(["year", "food_code"])
print(prices.index.levels[0])

# we'll focus on the latest price data
prices = prices.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices = prices.dropna(subset="price")

print(f"We have prices for {prices.shape[0]} unique recipes (FNDDS food codes)")

## Dietary Requirements



As before, we'll get our dietary requirements from the USDA.



In [1]:
rda = read_sheets(url, sheet="rda")

rda = rda.set_index("Nutrient")

rda.columns

## Putting It All Together



Earlier, we generated a dataframe of foods and nutrients. This included something like 65 different nutrients over 8,900 recipes! Unfortunately, our price data far fewer foods, so we have to narrow the set of foods from which we are choosing. I'll solve this issue by taking the set intersection of the two sets of food codes, and then select those common food codes from both dataframes.



In [1]:
common_recipes = df.index.intersection(prices.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
df = df.loc[common_recipes]
prices = prices.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices.index = prices.index.map(food_names)

A_all = df.T

A<sub>all</sub> will have the same number of foods as p has prices, but we now must to trim down the number of nutrients to include only those for which we have constraints. We'll look at the shapes of all these objects to be sure that the matrix multiplication operations are well defined.



In [1]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Child_1_3', 'Female_4_8', 'Male_4_8', 'Female_9_13', 'Male_9_13', 
'Female_14_18', 'Male_14_18','Female_19_30', 'Male_19_30', 
'Female_31_50', 'Male_31_50', 'Female_51U', 'Male_51U']
'''
group = "Female_19_30"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_all.reindex(bmin.index).dropna(how='all')
Amax = A_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices.shape=}")

## Solving the Problem



First, we find a solution to the problem



In [1]:
from  scipy.optimize import linprog as lp
import numpy as np
p = prices
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
result = lp(p, -A, -b, method='highs')

result

Let's interpret this.  Start with the cost of the solution:



In [1]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Next, what is it we're actually eating?



In [1]:
# lets mess with the index on price df so they are recipe names not ids.

# get the result x in a series with food names
diet = pd.Series(result.x,index=prices.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(round(diet[diet >= tol], 2))

Given this diet, what are nutritional outcomes?



In [1]:
tab = pd.DataFrame({"Outcome":A.to_numpy()@diet.to_numpy(),"Recommendation":np.abs(b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)

Finally, what are the constraints that bind?  Finding a less expensive
diet might involve finding less expensive sources for these particular nutrients.



In [1]:
print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol].index.tolist())